In [1]:
import pandas as pd
from hazm import Lemmatizer, Stemmer
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from farsi_tools import stop_words
import demoji
import pickle
import numpy as np
from farsi_tools import stop_words
from hazm import word_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.naive_bayes import GaussianNB
lem = Lemmatizer()
ste = Stemmer()

**this is data from twitter with negative or positive**

In [2]:
dataset = pd.read_csv('Corpus/twitter.csv')[:2000]

**Droping Null raws and indexing again**

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fulltext  2000 non-null   object
 1   label     2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [4]:
dataset.isnull().sum()

fulltext    0
label       0
dtype: int64

In [5]:
dataset = dataset.dropna(axis=0)
dataset.reset_index(drop=True,inplace=True)
dataset['label'].astype(np.int64)

0       0
1       1
2       1
3       1
4       1
       ..
1995    1
1996    0
1997    1
1998    1
1999    1
Name: label, Length: 2000, dtype: int64

**Cleaning Text**

In [6]:
for i, j in dataset.iterrows():
    # iterating in row and return index and value
    # reading column
    t = j['fulltext']
    # removing urls from text
    text = re.sub(r'([@][\w]+)', '', t)
    # removing english word
    text = re.sub('[A-Za-z]', '', text)
    # removing external word
    text = re.sub('[ ]{2,}', '', text)
    # removing sepcial word
    text = re.sub(r'[!@#$%^&*(),.?":{}|<>/«»،؛؟…_\-+\\\u200c]', '', text)
    # removing arabic
    text = re.sub(r'[\u064E-\u0655\u06F0-\u06F9\u0660-\u0669]', '', text)
    text = re.sub(r'[\u2600-\u26FF0-9]', '', text)
    # change emoji to feeling
    text = demoji.replace(text)
    dataset.loc[i] = {'fulltext': ''.join(text), 'label': j['label']}

In [7]:
# detecting null value
dataset.isnull().sum()

fulltext    0
label       0
dtype: int64

In [8]:
#tokeniz,removing stop words,stem,lem
for r,i in enumerate(dataset['fulltext']):
    # tokenizing
    word = word_tokenize(i)
    # remove stopwords
    word = [i for i in word if i not in stop_words()]
    # stemming by using hazm
    word = [ste.stem(i) for i in word]
    # Lem by using hazm
    word = [lem.lemmatize(i).replace('#',' ') for i in word]
    # re filling dataset
    dataset.loc[r] = {'fulltext': ' '.join(word), 'label': dataset['label'][r]}

In [9]:
# separating text and label
y = dataset['label']
x=dataset['fulltext']

In [10]:
# splitting dataset for model
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [11]:
# encoding labels to 0,1,..
encoder = LabelEncoder()
y_lab=encoder.fit_transform(y_train)
#vectorizing text ngram 1,2,3
vec = TfidfVectorizer(sublinear_tf = True,
                      use_idf = True, ngram_range = (1, 3))
vec.fit(x_train)
x_lab=vec.transform(x_train).toarray()

In [12]:
# defining our model
clas=SVC(C=1,kernel='linear')
clas.fit(x_lab,y_lab)


SVC(C=1, kernel='linear')

In [13]:
# test model with test data
x_test_vec=vec.transform(x_test).toarray()
y_test_enc=encoder.transform(y_test)
y_pred=clas.predict(x_test_vec)

In [14]:
print(f"accuracy :{accuracy_score(y_test,y_pred)}\n",f'confusion_matrix {confusion_matrix(y_test,y_pred)}\n',f'report{classification_report(y_test,y_pred)}' )

accuracy :0.88
 confusion_matrix [[244  52]
 [ 20 284]]
 report              precision    recall  f1-score   support

           0       0.92      0.82      0.87       296
           1       0.85      0.93      0.89       304

    accuracy                           0.88       600
   macro avg       0.88      0.88      0.88       600
weighted avg       0.88      0.88      0.88       600



In [15]:
gnb=GaussianNB(var_smoothing=10)
gnb.fit(x_lab,y_lab)
g_p=gnb.predict(x_test_vec)
g_p=gnb.predict(x_test_vec)
print(accuracy_score(y_pred,g_p))

0.9366666666666666


In [20]:
pickle.dump(clas, open("12.pkl", "wb"))

In [24]:
def predict(text):
    text = re.sub(r'([@][\w]+)', '', text)
    text = re.sub('[A-Za-z]', '', text)
    text = re.sub('[ ]{2,}', '', text)
    text = re.sub(r'[!@#$%^&*(),.?":{}|<>/«»،؛؟…_\-+\\\u200c]', '', text)
    text = re.sub(r'[\u064E-\u0655\u06F0-\u06F9\u0660-\u0669]', '', text)
    text = re.sub(r'[\u2600-\u26FF0-9]', '', text)
    text = demoji.replace(text)
    word = word_tokenize(text)
    word = [i for i in word if i not in stop_words()]
    word = [ste.stem(i) for i in word]
    word = [lem.lemmatize(i).replace('#',' ') for i in word]
    word =' '.join(word)
    vect    =vec.transform([word]).toarray()
    predicted=clas.predict(vect)
    if predicted==1:
        return 'negative'
    else:
        return 'positive'


In [25]:
text=' همچنان دارین به آشغال بودنتون ادامه میدین شما دوتا؟'
predict(text)

'negative'

In [19]:
save=pickle.dump(clas,open('digi.pk','wb'))